# **Hull Tactical: RandomForest**


---

## 🌳 **RandomForestRegressor and Multi-Output Regression**

`RandomForestRegressor` (from `sklearn.ensemble`) natively supports **multi-output regression**.
This means you can pass a **2D target array** `y` with shape `(n_samples, n_outputs)` directly to `.fit()`.
Each output column is treated as a separate regression problem, but the model shares the same set of trees, making training efficient.


## **Key points**:

* Unlike LightGBM/XGBoost, **no wrapper is required** (`MultiOutputRegressor` is not needed).
* Predictions preserve the same dimensionality: if `y` has 3 target variables, the output will have 3 columns.
* It is especially useful when the targets are related (e.g., predicting multiple sensor readings at once).

---


In [ ]:
import os
from pathlib import Path
import datetime

from tqdm import tqdm
from dataclasses import dataclass, asdict

import polars as pl 
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

import kaggle_evaluation.default_inference_server

# ============ PATHS ============
DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

# ============ RETURNS TO SIGNAL CONFIGS ============
MIN_SIGNAL: float = 0.0                         # Minimum value for the daily signal 
MAX_SIGNAL: float = 2.0                         # Maximum value for the daily signal 
SIGNAL_MULTIPLIER: float = 400.0                # Multiplier of the OLS market forward excess returns predictions to signal 

# ============ RANDOM FOREST CONFIGS ============
N_ESTIMATORS: int = 100                         # Number of trees in the forest
MAX_DEPTH: int = None                           # Maximum depth of the tree
MIN_SAMPLES_SPLIT: int = 2                      # Minimum number of samples required to split an internal node
MIN_SAMPLES_LEAF: int = 1                       # Minimum number of samples required to be at a leaf node
RANDOM_STATE: int = 42                          # Random state for reproducibility

@dataclass
class DatasetOutput:
    X_train : pl.DataFrame 
    X_test: pl.DataFrame
    y_train: pl.Series
    y_test: pl.Series
    scaler: StandardScaler

@dataclass 
class RandomForestParameters:
    n_estimators : int 
    max_depth: int
    min_samples_split: int
    min_samples_leaf: int
    random_state: int
    
    def __post_init__(self): 
        if self.n_estimators <= 0:
            raise ValueError("n_estimators must be greater than 0")
        
@dataclass(frozen=True)
class RetToSignalParameters:
    signal_multiplier: float 
    min_signal : float = MIN_SIGNAL
    max_signal : float = MAX_SIGNAL

# Set the Parameters
ret_signal_params = RetToSignalParameters(
    signal_multiplier= SIGNAL_MULTIPLIER
)

rf_params = RandomForestParameters(
    n_estimators = N_ESTIMATORS,
    max_depth = MAX_DEPTH,
    min_samples_split = MIN_SAMPLES_SPLIT,
    min_samples_leaf = MIN_SAMPLES_LEAF,
    random_state = RANDOM_STATE
)

def load_trainset() -> pl.DataFrame:
    """
    Loads and preprocesses the training dataset.
    """
    return (
        pl.read_csv(DATA_PATH / "train.csv")
        .rename({'market_forward_excess_returns':'target'})
        .with_columns(
            pl.exclude('date_id').cast(pl.Float64, strict=False)
        )
        .head(-10)
    )

def load_testset() -> pl.DataFrame:
    """
    Loads and preprocesses the testing dataset.
    """
    return (
        pl.read_csv(DATA_PATH / "test.csv")
        .rename({'lagged_forward_returns':'target'})
        .with_columns(
            pl.exclude('date_id').cast(pl.Float64, strict=False)
        )
    )

def create_example_dataset(df: pl.DataFrame) -> pl.DataFrame:
    """
    Creates new features and cleans a DataFrame.
    """
    vars_to_keep = [
        "S2", "E2", "E3", "P9", "S1", "S5", "I2", "P8",
        "P10", "P12", "P13", "U1", "U2"
    ]

    return (
        df.with_columns(
            (pl.col("I2") - pl.col("I1")).alias("U1"),
            (pl.col("M11") / ((pl.col("I2") + pl.col("I9") + pl.col("I7")) / 3)).alias("U2")
        )
        .select(["date_id", "target"] + vars_to_keep)
        .with_columns([
            pl.col(col).fill_null(pl.col(col).ewm_mean(com=0.5))
            for col in vars_to_keep
        ])
        .drop_nulls()
    )
    
def join_train_test_dataframes(train: pl.DataFrame, test: pl.DataFrame) -> pl.DataFrame:
    """
    Joins two dataframes by common columns and concatenates them vertically.
    """
    common_columns = [col for col in train.columns if col in test.columns]
    
    return pl.concat([train.select(common_columns), test.select(common_columns)], how="vertical")

def split_dataset(train: pl.DataFrame, test: pl.DataFrame, features: list) -> DatasetOutput: 
    """
    Splits the data into features (X) and target (y), and scales the features.
    """
    X_train = train.drop(['date_id','target']) 
    y_train = train.get_column('target')
    X_test = test.drop(['date_id','target']) 
    y_test = test.get_column('target')
    
    scaler = StandardScaler() 
    
    X_train_scaled_np = scaler.fit_transform(X_train)
    X_train = pl.from_numpy(X_train_scaled_np, schema=features)
    
    X_test_scaled_np = scaler.transform(X_test)
    X_test = pl.from_numpy(X_test_scaled_np, schema=features)
    
    return DatasetOutput(
        X_train = X_train,
        y_train = y_train, 
        X_test = X_test, 
        y_test = y_test,
        scaler = scaler
    )

def convert_ret_to_signal(
    ret_arr: np.ndarray,
    params: RetToSignalParameters
) -> np.ndarray:
    """
    Converts raw model predictions (expected returns) into a trading signal.
    """
    return np.clip(
        ret_arr * params.signal_multiplier + 1, params.min_signal, params.max_signal
    )

# Looking at the Data
train: pl.DataFrame = load_trainset()
test: pl.DataFrame = load_testset() 
print(train.tail(3)) 
print(test.head(3))

# Generating the Train and Test
df: pl.DataFrame = join_train_test_dataframes(train, test)
df = create_example_dataset(df=df) 
train: pl.DataFrame = df.filter(pl.col('date_id').is_in(train.get_column('date_id')))
test: pl.DataFrame = df.filter(pl.col('date_id').is_in(test.get_column('date_id')))

FEATURES = [col for col in test.columns if col not in ['date_id', 'target']]

dataset: DatasetOutput = split_dataset(train=train, test=test, features=FEATURES) 

X_train: pl.DataFrame = dataset.X_train
X_test: pl.DataFrame = dataset.X_test
y_train: pl.DataFrame = dataset.y_train
y_test: pl.DataFrame = dataset.y_test
scaler: StandardScaler = dataset.scaler 

# Fitting the Random Forest Model
model = RandomForestRegressor(**asdict(rf_params))
model.fit(X_train.to_pandas(), y_train.to_pandas())

print(f"RandomForestRegressor fitted with {rf_params.n_estimators} estimators")

# Prediction Function via Kaggle Server
def predict(test: pl.DataFrame) -> float:
    test = test.rename({'lagged_forward_returns':'target'})
    df: pl.DataFrame = create_example_dataset(test)
    X_test: pl.DataFrame = df.select(FEATURES)
    X_test_scaled_np: np.ndarray = scaler.transform(X_test)
    X_test_scaled = pl.from_numpy(X_test_scaled_np, schema=FEATURES)
    raw_pred: float = model.predict(X_test_scaled.to_pandas())[0]
    return convert_ret_to_signal(raw_pred, ret_signal_params)

# Launch Server
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))